# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686591


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:31,  3.27s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:03,  2.36s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.77s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.30s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:13<00:01,  3.58it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.77it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.04it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.33it/s]

smithkurtis                           0.000679
Slayz                                 0.000307
Petrusperes                           0.000604
Camelron                              0.000343
asucich                               0.000469
djswanso                              0.000539
kjkavene                              0.000371
jmfranci                              0.000822
tosnyder                              0.000843
not-logged-in-2c1d34aa84b01548ac93    0.000746
not-logged-in-746dafd91ba51764d0ed    0.005701
ferlin1998                            0.001424
MilesGray                             0.008967
gabrielweiss                          0.083586
pangeli5                              0.000629
Planetme                              0.003330
not-logged-in-214814e52004b42491a4    0.062171
olafvoss                              0.000320
aidenr                                0.000289
acapirala                             0.000606
jlj080498                             0.001027
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())